# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [25]:
#!pip install chromadb

^C


In [26]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [27]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [73]:
llm = OpenAI(temperature=0)

In [32]:
from langchain_community.document_loaders import WebBaseLoader

In [74]:
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=0)
embeddings = OpenAIEmbeddings()


In [75]:
loader = WebBaseLoader("https://www.trailandkale.com/tips/beginners-guide-trail-running/")

In [76]:
docs = loader.load()
trail_running_texts = text_splitter.split_documents(docs)
trail_running_db = Chroma.from_documents(trail_running_texts, embeddings, collection_name="barefoot")
trail_running = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=trail_running_db.as_retriever()
)

Created a chunk of size 7294, which is longer than the specified 3000
Created a chunk of size 3320, which is longer than the specified 3000
Created a chunk of size 4426, which is longer than the specified 3000
Created a chunk of size 3111, which is longer than the specified 3000


In [77]:
loader = WebBaseLoader("https://anyasreviews.com/barefoot-shoes-faq/")

In [78]:
docs = loader.load()
barefoot_texts = text_splitter.split_documents(docs)
barefoot_db = Chroma.from_documents(barefoot_texts, embeddings, collection_name="barefoot")
barefoot = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=barefoot_db.as_retriever()
)

In [79]:
#!pip install beautifulsoup4


## Create the Agent

In [80]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [81]:
tools = [
    Tool(
        name="Trail running QA System",
        func=trail_running.run,
        description="useful for when you need to answer questions about trail running. Input should be a fully formed question.",
    ),
    Tool(
        name="barefoot QA System",
        func=barefoot.run,
        description="useful for when you need to answer questions about barefoot lifestyle. Input should be a fully formed question.",
    ),
]

In [82]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [83]:
agent.invoke(
    "What is needed for trail running?"
)



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Trail running QA System
Action Input: What is needed for trail running?
Observation:  Trail running shoes and a hydration pack or race vest are recommended as the top two pieces of gear for trail running. Other gear, such as trail running poles, GPS watches, hats, and sunglasses, can also be helpful depending on the individual's preferences and the type of trail running they do.
Thought: I now know the final answer
Final Answer: The top two pieces of gear recommended for trail running are trail running shoes and a hydration pack or race vest. Other gear, such as trail running poles, GPS watches, hats, and sunglasses, can also be helpful depending on the individual's preferences and the type of trail running they do.

> Finished chain.


{'input': 'What is needed for trail running?',
 'output': "The top two pieces of gear recommended for trail running are trail running shoes and a hydration pack or race vest. Other gear, such as trail running poles, GPS watches, hats, and sunglasses, can also be helpful depending on the individual's preferences and the type of trail running they do."}

In [84]:
agent.invoke("are there any benefits on trying to go barefoot?")



> Entering new AgentExecutor chain...
 Going barefoot may have some benefits, but it's important to consider the potential risks as well.
Action: barefoot QA System
Action Input: "What are the benefits of going barefoot?"
Observation: 

There are several potential benefits of going barefoot, including improved foot strength and mobility, better balance and posture, and reduced risk of foot and ankle injuries. Going barefoot can also help to improve sensory feedback and proprioception (awareness of body position and movement) in the feet. Additionally, some people find that going barefoot can help to alleviate foot pain and discomfort caused by wearing traditional shoes.
Thought: It's important to also consider the potential risks of going barefoot, such as increased risk of cuts, scrapes, and infections, as well as potential discomfort on rough or hot surfaces.
Action: barefoot QA System
Action Input: "What are the risks of going barefoot?"
Observation: 
The risks of going barefoot i

{'input': 'are there any benefits on trying to go barefoot?',
 'output': "Ultimately, the decision to go barefoot should be based on personal preference and individual needs. It's important to weigh the potential benefits and risks and make a decision that is right for you."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [85]:
tools = [
    Tool(
        name="Trail running QA System",
        func=trail_running.run,
        description="useful for when you need to answer questions about trail running. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="barefoot QA System",
        func=barefoot.run,
        description="useful for when you need to answer questions about barefoot lifestyle. Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [86]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [87]:
agent.invoke(
    "Is trail running something everyone can do?"
)



> Entering new AgentExecutor chain...
 It depends on a person's physical abilities and health.
Action: Trail running QA System
Action Input: "Is trail running something everyone can do?"
Observation: 
Yes, trail running is something that anyone can do, as long as they are physically able and have the proper gear and preparation. However, it is important to start slowly and build up mileage and experience gradually to avoid injury and ensure a positive experience.


> Finished chain.


{'input': 'Is trail running something everyone can do?',
 'output': '\nYes, trail running is something that anyone can do, as long as they are physically able and have the proper gear and preparation. However, it is important to start slowly and build up mileage and experience gradually to avoid injury and ensure a positive experience.'}

In [88]:
agent.invoke("How is going barefoot going to impact me negatively?")



> Entering new AgentExecutor chain...
 You should always think about the potential negative consequences of any lifestyle change.
Action: barefoot QA System
Action Input: "How will going barefoot affect my health?"
Observation: 

Going barefoot can have a positive impact on your health in several ways. It can improve your foot strength, mobility, and stability, which can help prevent foot pain and injuries. It can also improve your overall body alignment, which can alleviate back and neck pain. Additionally, going barefoot can provide sensory stimulation for your feet, which can improve balance and proprioception. However, it is important to transition gradually and listen to your body to avoid overuse injuries. It is also important to maintain good hygiene and avoid potential hazards when going barefoot.


> Finished chain.


{'input': 'How is going barefoot going to impact me negatively?',
 'output': '\n\nGoing barefoot can have a positive impact on your health in several ways. It can improve your foot strength, mobility, and stability, which can help prevent foot pain and injuries. It can also improve your overall body alignment, which can alleviate back and neck pain. Additionally, going barefoot can provide sensory stimulation for your feet, which can improve balance and proprioception. However, it is important to transition gradually and listen to your body to avoid overuse injuries. It is also important to maintain good hygiene and avoid potential hazards when going barefoot.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [89]:
tools = [
    Tool(
        name="Trail running QA System",
        func=trail_running.run,
        description="useful for when you need to answer questions about trail running. Input should be a fully formed question.",
    ),
    Tool(
        name="barefoot QA System",
        func=barefoot.run,
        description="useful for when you need to answer questions about barefoot lifestyle. Input should be a fully formed question.",
    ),
]

In [90]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [91]:
agent.invoke(
    "What should I keep in mind if I want to go trail running but try doing it barefoot?"
)



> Entering new AgentExecutor chain...
 You should consider the potential risks and benefits of barefoot trail running.
Action: Trail running QA System
Action Input: "What are the risks and benefits of barefoot trail running?"
Observation: 

The risks of barefoot trail running include potential injuries to the feet, such as cuts, scrapes, and bruises from rough terrain. There is also a risk of overuse injuries if the runner is not properly conditioned and prepared for barefoot running.

However, there are also many benefits to barefoot trail running. It can improve foot strength and balance, as well as promote a more natural running form. It can also reduce the risk of certain injuries, such as knee and ankle injuries, that may occur while running in traditional shoes. Additionally, barefoot trail running can be a more enjoyable and sensory experience, allowing the runner to connect with nature and their surroundings.
Thought: I now know the final answer
Final Answer: The risks of bar

{'input': 'What should I keep in mind if I want to go trail running but try doing it barefoot?',
 'output': 'The risks of barefoot trail running include potential injuries to the feet, while the benefits include improved foot strength and balance, reduced risk of certain injuries, and a more enjoyable sensory experience.'}